In [54]:
import psycopg2, re, psycopg2
import pandas as pd
from datetime import datetime
import random

In [4]:
df = pd.read_csv("./dentist_rand_sched_utf.csv")

In [6]:
df.head()

,dentist_name,clinic_name,address,city_town,province,region,zip_code,contact_number,Monday_start,Monday_end,...,Wednesday_end,Thursday_start,Thursday_end,Friday_start,Friday_end,Saturday_start,Saturday_end,Sunday_start,Sunday_end,id
0,AIDA MAITIM JALUAG,A.M. JALUAG DENTAL CLINIC,"G/F UNIT B-3 LEGASPI TOWER 300, 2600 ROXAS BLV...",MANILA,METRO MANILA,NCR,1004,5313-8506 / (0905) 433-1147,8,17,...,17,8,17,8,17,NaN,NaN,NaN,NaN,4838
1,ANGE LUMAWAG,L & S DENTAL CARE CLINIC,"7/F VIP BLDG., 1140 ROXAS BOULEVARD, ERMITA",MANILA,METRO MANILA,NCR,1000,7617-7079 / (0917) 546-6292,9,18,...,18,9,18,9,18,8.0,17.0,8.0,17.0,4839
2,MARJORIE JOY LIM,DENTAL WELLNESS STUDIO,"UNIT GD BURGUNDY WEST BAY TOWER, 820 PABLO OCA...",MANILA,METRO MANILA,NCR,1004,5313-5746 / (0998) 573-3788,10,19,...,19,10,19,10,19,7.0,11.0,7.0,11.0,4840
3,CONCEPCION PONCE PEÑAFLORIDA,PONCE PEÑAFLORIDA DENTAL CLINIC,"RM. 1, 2/F ARV BLDG., 1728 RIZAL AVENUE, STA. ...",MANILA,METRO MANILA,NCR,1014,8741-7260 / 7577-1833 / (0917) 516-1066,8,17,...,17,8,17,8,17,9.0,13.0,9.0,13.0,4841
4,MARISSA LOMOTAN,MARISSA ORIO-LOMOTAN DENTAL CENTRAL,4747 OLD STA. MESA,MANILA,METRO MANILA,NCR,1016,7219-3617 / (0917) 147-5214,10,19,...,19,10,19,10,19,NaN,NaN,NaN,NaN,4842


In [25]:
# get user's location and search all dentist in that area
user_city = "caloocan"
# display clinics in that area
dentists = df.loc[df['city_town'].str.contains(user_city.lower(),case=False)]
dentists = dentists['clinic_name'].to_string(index=False)
dentist_list = dentists.strip().replace('\\n',"\n")
print(dentist_list)

         PUJALTE DENTAL CLINIC
         EZETHEA DENTAL CLINIC
           C. ASTOR DENTAL HUB
     TOOTH CHECK DENTAL CLINIC
      MJCR ORTHO DENTAL CLINIC
       GC TEJADA DENTAL CLINIC
           RJKDC DENTAL CLINIC
DRK NAVARRO DENTAL CARE CLINIC


In [26]:
# choose what clinic and return id of selected clinic
clinic_name = "PUJALTE DENTAL CLINIC"
clinic_id = df.loc[df['clinic_name'].str.contains(clinic_name.lower(),case=False)]['id'].to_string(index=False)
print(clinic_id)

4849


In [57]:
# select date of appointment
date_today = datetime.now()
user_selected = "12/24/2023" # accepts mm/dd/yyyy format only for now, use date picker if available
date_format = '%m/%d/%Y'
user_date = datetime.strptime(user_selected, date_format)

# check if date is done or not
if user_date < date_today:
    print("cant select that date")
else:
    # check if selected clinic can accomodate that day
    sch_day = user_date.strftime('%A')
    time_start = df.loc[df['id'] == int(clinic_id)][f'{sch_day}_start'].values
    print(time_start)
    can_accomodate = time_start[0]>0

[9.]


In [61]:
# select time of appointment (in military time)
user_time = 9
user_id = random.randint(0,20) # wala ako access haha
if can_accomodate:
    # making sure selected time is within operating hours
    if time_start[0] > user_time:
        print("selected time is outside of clinic hours")
    else:
        selected_start_time = user_time
        stop_time = selected_start_time + 1

# convert everything to a sql statement:
sql_query = f'''INSERT INTO app_dentist_schedule (clinic_id, user_id, appointment_date, "start", stop) VALUES ({clinic_id}, {user_id}, '{user_date.date().strftime('%Y-%m-%d')}', {selected_start_time}, {stop_time})'''
print(sql_query)


INSERT INTO app_dentist_schedule (clinic_id, user_id, appointment_date, "start", stop) VALUES (4849, 7, '2023-12-24', 9, 10)
